I need Chemical Formula, Cell Parameters, Space Group

In [58]:
import plotly.graph_objects as go

def plotting_struct(structure):
    x, y, z = structure[:,0] , structure[:,1], structure[:,2]
    fig = go.Figure(data=[go.Scatter3d(
        x = x,
        y = y,
        z = z,
        mode='markers',
        marker=dict(
            size=5,
            opacity=0.8
        )
    )])
    
    fig.update_layout(
        scene=dict(
            xaxis=dict(title='X'),
            yaxis=dict(title='Y'),
            zaxis=dict(title='Z')
        )
    )
    fig.show()

In [74]:
from pymatgen.io.cif import CifParser

# Specify the path to your CIF file
cif_path = "ICSD_CollCode257256.cif"

# Create a CIF parser and parse the CIF file
parser = CifParser(cif_path)

structure = parser.parse_structures(primitive=True)[0]

# Print information about the crystal structure
print("Lattice Parameters:")
print("a =", structure.lattice.a)
print("b =", structure.lattice.b)
print("c =", structure.lattice.c)
print("alpha =", structure.lattice.alpha)
print("beta =", structure.lattice.beta)
print("gamma =", structure.lattice.gamma)
print()

print("Space Group:", structure.get_space_group_info())
print()

print("Asymmetric Unit:")
for site in structure.sites:
    print(f"{site.specie}: {site.frac_coords}")


Lattice Parameters:
a = 4.0848
b = 4.0848
c = 4.0848
alpha = 90.0
beta = 90.0
gamma = 90.0

Space Group: ('Pm-3m', 221)

Asymmetric Unit:
Cs+: [0. 0. 0.]
Cl-: [0.5 0.5 0.5]


In [97]:
structure = parser.parse_structures(primitive=True)[0]

unit_cell = structure.make_supercell(2)

unit_cell_coordinates = unit_cell.cart_coords
plotting_struct(unit_cell_coordinates)

In [99]:

condition = unit_cell_coordinates <= 4.0849
# If you want to keep the original shape by replacing values that don't satisfy the condition with NaNs:
unit_cell_coordinates[~condition] = np.nan

plotting_struct(unit_cell_coordinates)

# Find a way to make this work if the values are different for x y and z

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [78]:
import numpy as np

a_atoms = 2
b_atoms = 2
c_atoms = 2

vertices = []

a_dist = structure.lattice.a
b_dist = structure.lattice.b
c_dist = structure.lattice.c

alpha = structure.lattice.alpha
beta = structure.lattice.beta
gamma = structure.lattice.gamma

for a in range(a_atoms):
        for b in range(b_atoms):
            for c in range(c_atoms):
                vertices.append([(a * a_dist) + (np.cos(np.deg2rad(180 - alpha)) * b),
                                 (b * b_dist) + (np.cos(np.deg2rad(180 - beta)) * c),
                                 (c * c_dist) + (np.cos(np.deg2rad(180 - gamma)) * a)])  
                
vertices = np.array(vertices)
vertices[np.abs(vertices) < 1e-10] = 0

In [81]:
corners = np.array([np.min(vertices, axis=0), np.max(vertices, axis=0)])
print(corners)

# Check if the values in unit cell coordinates are within the vertices cube
within_cube = np.all((unit_cell_coordinates >= corners[0]) & (unit_cell_coordinates <= corners[1]), axis=1)

# Print the result
print(within_cube)


[[0.     0.     0.    ]
 [4.0848 4.0848 4.0848]]
[ True  True False  True  True False False False False  True  True False
  True False False False False False False False False False False False
 False False False  True False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False]


In [79]:
[(a_atoms * a_dist) + (np.cos(np.deg2rad(180 - alpha)) * b),
(b_atoms * b_dist) + (np.cos(np.deg2rad(180 - beta)) * c),
(c_atoms* c_dist) + (np.cos(np.deg2rad(180 - gamma)) * a)]

[8.1696, 8.1696, 8.1696]

In [76]:
plotting_struct(vertices)